# 部署HuggingFace模型

HuggingFace是一个开源的模型社区，机器学习开发者在社区中可以分享、发现和使用各类机器学习模型。

本文将介绍如何将HuggingFace社区的模型部署到PAI创建模型推理服务。


## 安装和配置SDK


我们需要首先安装PAI Python SDK以运行本示例。

In [ ]:

!python -m pip install --upgrade alipai




SDK需要配置访问阿里云服务需要的AccessKey，以及当前使用的工作空间和OSS Bucket。在PAI SDK安装之后，通过在**命令行终端** 中执行以下命令，按照引导配置密钥、工作空间等信息。


```shell

# 以下命令，请在 命令行终端 中执行.

python -m pai.toolkit.config

```

我们可以通过以下代码验证配置是否已生效。

In [ ]:
import pai
from pai.session import get_default_session

print(pai.__version__)

sess = get_default_session()

# 获取配置的工作空间信息
assert sess.workspace_name is not None
print(sess.workspace_name)

## 部署HuggingFace模型

在本示例中，我们将使用HuggingFace社区提供的情感分类模型 [distilbert-base-uncased-finetuned-sst-2-english](https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)部署一个模型在线服务，他支持将一段英文文本分类为正面或负面情感。

通过相应的[模型的详情页](https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english/tree/main)，我们可以获取部署模型所需的信息，包括模型ID(``MODEL_ID``)、模型任务类型(``TASK``)、模型版本(``REVISION``)。

![](../../images/huggingface-model.png)


通过PAI Python SDK提供的``HuggingFaceModel``，我们可以轻松地将HuggingFace社区的模型部署到PAI上。

In [ ]:
from pai.huggingface import HuggingFaceModel


# 初始化一个HuggingFaceModel
m = HuggingFaceModel(
    command="python app.py",  # 模型服务启动命令
    transformers_version="latest",  # 使用的transformers版本, 'latest'表示使用PAI目前支持的最新的版本
    environment_variables={
        "MODEL_ID": "distilbert-base-uncased-finetuned-sst-2-english",  # 部署模型的ID
        "TASK": "text-classification",  # 部署的模型任务类型
        "REVISION": "main",  # 部署模型的版本信息
    },
)

In [ ]:
from pai.common.utils import random_str


# 部署模型，创建一个模型在线服务
p = m.deploy(
    service_name=f"hf_model_deploy_{random_str(n=8)}",  # 模型服务的名称（地域内唯一）
    instance_type="ecs.g6.large",  # 模型服务使用的机器实例规格
    options={
        "enable_webservice": True,  # 以AIWeb应用的模式启动，支持用户在Web浏览器上使用模型在线服务
    },
)

In [ ]:
p.predict(data={"data": ["I love you"]})

测试完成之后，删除服务，释放机器资源。

In [ ]:
p.delete_service()